In [2]:
#import statements
import datetime
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

#from keypass import NOAA_api
import pylab as pl
from IPython import display

from functools import partial
import pyproj
from sklearn import datasets
from sklearn.tree import DecisionTreeRegressor
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score,explained_variance_score
from numpy import absolute,mean,std

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import interactive


### $TCO_2$ was filled in. Now load the dataset for some visuals. 

Every 25th row is saved to make the dashboard useable (ie faster). 

Len before 1218966 and after trim 48759

there are 33 cruises with <100 datapoints out of 936 cruises so I consider this data loss acceptable 

In [ ]:
#https://stackoverflow.com/questions/60609760/is-there-a-way-to-import-csv-file-from-github-automatically-to-my-jupyter-notebo (accessed Oct. 08, 2020).


#df =pd.read_csv('C:\\Users\\ossan\\DATA606\\DATA\\TCO2_filled.csv',index_col=0) #ORIGINAL
#plotly is too slow with complete set -use this for now 
url = 'https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/data/TCO2_small.csv'
df = pd.read_csv(url,index_col=0)
dftrim = df 
#print(len(df))
df['year'] =pd.to_numeric( df['year'])
df['year'] =df['year'].astype(int)
g=df.groupby('year')

In [ ]:
#dftrim = df.iloc[::25,:]
#print('len before '+ str(len(df))+' and after trim for plotly '+str(len(dftrim)))
#dftrim.to_csv( 'C:\\Users\\ossan\\DATA606\\DATA\\TCO2_small.csv') #save metrics 

In [ ]:
frames = {}
for x,y in g:
    frames[x] = y
#frames[list(frames.keys())[-1]]

In [ ]:

var = df.columns 
years = list(frames.keys())
list_updatemenus=[];temp_dict={}
for n, year in enumerate(years):
    visible = [False] * len(years)
    visible[n] = True
    temp_dict = dict(label = str(year),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': 'Year %d' % year}])

    list_updatemenus.append(temp_dict)
    
list_c=[];c_dict={}
for n, year in enumerate(var):
    visible = [False] * len(var)
    visible[n] = True
    c_dict = dict(label = str(var),
                 method = 'update',
                 args = [{'visible': visible},
                         {'title': var}])

    list_c.append(c_dict)
    

## Select the dataset variable column and year using the dropdowns. 

In [ ]:
#fig5 = go.Figure()
col=['Cruise', 'Station',  'Year', 'Month', 'Day', 'Bottom Depth', 'Max Sample Depth', 'Pressure', 'Depth',
       'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2']

actu=['cruise', 'station', 'year', 'month', 'day', 'bottomdepth', 'maxsampdepth', 'pressure', 'depth',
       'temperature', 'theta', 'salinity', 'sigma0', 'sigma1', 'sigma2',
       'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate', 'nitrite', 'silicate',
       'phosphate', 'talk', 'phts25p0', 'phtsinsitutp', 'tco2']
def plot_compare_lag(var, year):

    #data = te(s,BalancingAuthority)
    fig5 =go.Figure(go.Scattergeo(locationmode ='ISO-3',lon=dftrim["longitude"],
                                  lat=dftrim["latitude"],#color=var,
        mode ='markers',marker = dict(
            size = 2, color = dftrim[actu[col.index(var)]], colorscale = 'Inferno',
            cmax = df[actu[col.index(var)]].max(),colorbar_title=var) ),
    layout= dict(title ='Variable: ' +var + ' | Year:'  + str(year.astype(str)[0:4]))  )
    fig5.show()
    return fig5 

interactive(plot_compare_lag, var=col, 
            year=list(np.linspace(1984,2019,(2019-1983))),  )


### See the changes over the years with this visual. The dataset is grouped by latitude and longitude and averaged per year. The latitude and longitude is rounded to the nearest ones place to allow for merging across cruise locations. 

In [4]:
url = 'https://raw.githubusercontent.com/ossana1/DATA606_FinalProject/master/data/Groupedlatlongyr.csv'
latlong= pd.read_csv(url,index_col=0)
latlong.drop(columns=['cruise','station','day','month','Fill','cast'],inplace=True)
#clean up columns for labels 
colu=['Year','Bottom Depth', 'Max Sample Depth', 'Pressure', 'Depth', 'Potential Temperature', 'theta', 'Salinity', 'Potential Temp 0m', 'Potential Temp 1km', 'Potential Temp 2km',
       'Potential Temp 3km', 'Neutral Density', 'oxygen', 'Actual O2 Utilization', 'Nitrate', 'Nitrite', 'Silicate',
       'Phosphate', 'Total Alkalinity', 'pH at STP', 'pH in Situ', 'tco2','Binned Depth']
actua=[ 'year', 'bottomdepth', 'maxsampdepth','pressure', 'depth', 'temperature', 'theta', 'salinity', 'sigma0',
       'sigma1', 'sigma2', 'sigma3', 'gamma', 'oxygen', 'aou', 'nitrate','nitrite', 'silicate', 'phosphate', 'talk', 'phts25p0', 'phtsinsitutp',
       'tco2', 'depthBinned']

#print(len(col),len(actu))

In [10]:
%matplotlib inline

from sklearn.neighbors import BallTree

def chang(var,start,end,df):
    start_year=latlong[latlong.year ==start].reset_index(); end_year=latlong[latlong.year ==end].reset_index()
    query_lats = end_year.latitude;query_lons = end_year.longitude
    
    varx = actua[colu.index(var)] 
    #search for closest lat and long 
    ###https://stackoverflow.com/questions/10549402/kdtree-for-longitude-latitude
    bt = BallTree(np.deg2rad(start_year[['latitude', 'longitude']].values), metric='haversine')
    distances, indices = bt.query(np.deg2rad(np.c_[query_lats, query_lons]))

    i = pd.DataFrame(list(zip(distances,indices)),columns=['distances','indices'])
    end_year['compare']  =i.iloc[:,1].astype(int)
    end_year['data'] = start_year.loc[end_year.compare,varx].values
    end_year['diff'] =(end_year[varx]- end_year['data'])
    return end_year

def plot_compare_lag(var, startyear,endyear):
    change = chang(var,startyear,endyear,latlong)
    fig =go.Figure(go.Scattergeo(locationmode ='ISO-3',lon=change["longitude"],
                                  lat=change["latitude"],#color=var,
        mode ='markers',marker = dict(
            size = 4, color = change['diff'], colorscale = 'viridis',
            colorbar_title=var) ), #cmax = 1.5*change['diff'].max(),
    layout= dict(title ='Variable Change: ' +var + ' | Start Year:'  + 
                 str(startyear.astype(str)[0:4]) +'| End Year:'  + 
                 str(endyear.astype(str)[0:4]))  )
    fig.show()
    return fig 



interactive(plot_compare_lag, var=colu, startyear=list(np.linspace(1984,2019,(2019-1983))), 
            endyear=list(np.linspace(1984,2019,(2019-1983))),)


interactive(plot_compare_lag, var=colu, startyear=list(np.linspace(1984,2019,(2019-1983))), 
            endyear=list(np.linspace(1984,2019,(2019-1983))),)


interactive(children=(Dropdown(description='var', options=('Year', 'Bottom Depth', 'Max Sample Depth', 'Pressu…